# In motion dataset SVM

In [ ]:
%run -i utils.py

# ECG

### SVM ECG FILTERED

In [ ]:
ecg = pd.read_csv("../DATA_CSV/ECG_filtered_noscaled_cammina.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ecg_df, test_ecg_df = train_test_split(ecg, test_size=0.2, stratify=ecg['Label'], random_state=123)

ecg_mat_train, label_ecg_train = create_array(train_ecg_df) 
ecg_mat_test, label_ecg_test  = create_array(test_ecg_df) 

X_train = ecg_mat_train
y_train = label_ecg_train
X_test = ecg_mat_test
y_test = label_ecg_test

In [ ]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with higher degree kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of higher degree kernel
degree = 4

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=degree), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=', degree, ' KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
#parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
#clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf = RandomizedSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='poly',degree=4, C=0.05, gamma=0.05).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

### SVM ECG FILTERED PCA

In [ ]:
ecg = pd.read_csv("../DATA_CSV/ECG_filtered_noscaled_cammina.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ecg_df, test_ecg_df = train_test_split(ecg, test_size=0.2, stratify=ecg['Label'], random_state=123)

ecg_mat_train, label_ecg_train = create_array(train_ecg_df) 
ecg_mat_test, label_ecg_test  = create_array(test_ecg_df) 

num_components = 0.95
pca2 = PCA(n_components = num_components)
pca2.fit(ecg_mat_train)
ecg_mat_train = pca2.transform(ecg_mat_train)
ecg_mat_test = pca2.transform(ecg_mat_test)

X_train = ecg_mat_train
y_train = label_ecg_train
X_test = ecg_mat_test
y_test = label_ecg_test

# from sklearn.preprocessing import MinMaxScaler
# scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
# X_train = scaling.transform(X_train)
# X_test = scaling.transform(X_test)

In [ ]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with higher degree kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of higher degree kernel
degree = 4

svc = SVC()
#clf = GridSearchCV(SVC(kernel='poly', degree=degree), parameters, cv=4, return_train_score=True)
clf = RandomizedSearchCV(SVC(kernel='poly', degree=degree, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=', degree, ' KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
#clf = GridSearchCV(SVC(kernel='rbf'), parameters, cv=4, return_train_score=True)
clf = RandomizedSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=500, gamma=0.05).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

### SVM ECG nonFILTERED

In [ ]:
ecg = pd.read_csv("../DATA_CSV/ECG_nofiltered_noscaled_cammina.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ecg_df, test_ecg_df = train_test_split(ecg, test_size=0.2, stratify=ecg['Label'], random_state=123)

ecg_mat_train, label_ecg_train = create_array(train_ecg_df) 
ecg_mat_test, label_ecg_test  = create_array(test_ecg_df) 

X_train = ecg_mat_train
y_train = label_ecg_train
X_test = ecg_mat_test
y_test = label_ecg_test

# from sklearn.preprocessing import MinMaxScaler
# scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
# X_train = scaling.transform(X_train)
# X_test = scaling.transform(X_test)

In [ ]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with higher degree kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of higher degree kernel
degree = 4

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=degree), parameters, cv=4, return_train_score=True)
#clf = RandomizedSearchCV(SVC(kernel='poly', degree=degree, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=', degree, ' KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='poly',degree=4, C=0.05, gamma=0.05).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

### SVM ECG noFILTERED PCA

In [ ]:
ecg = pd.read_csv("../DATA_CSV/ECG_nofiltered_noscaled_cammina.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ecg_df, test_ecg_df = train_test_split(ecg, test_size=0.2, stratify=ecg['Label'], random_state=123)

ecg_mat_train, label_ecg_train = create_array(train_ecg_df) 
ecg_mat_test, label_ecg_test  = create_array(test_ecg_df) 

num_components = 0.95
pca2 = PCA(n_components = num_components)
pca2.fit(ecg_mat_train)
ecg_mat_train = pca2.transform(ecg_mat_train)
ecg_mat_test = pca2.transform(ecg_mat_test)

X_train = ecg_mat_train
y_train = label_ecg_train
X_test = ecg_mat_test
y_test = label_ecg_test

# from sklearn.preprocessing import MinMaxScaler
# scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
# X_train = scaling.transform(X_train)
# X_test = scaling.transform(X_test)

In [ ]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with higher degree kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of higher degree kernel
degree = 4

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=degree, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=', degree, ' KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=50, gamma=0.05).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

# PPG

### SVM PPG FILTERED

In [ ]:
ppg = pd.read_csv("../DATA_CSV/PPG_filtered_noscaled_cammina.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ppg_df, test_ppg_df = train_test_split(ppg, test_size=0.2, stratify=ppg['Label'], random_state=123)

ppg_mat_train, label_ppg_train = create_array(train_ppg_df) 
ppg_mat_test, label_ppg_test   = create_array(test_ppg_df) 

X_train = ppg_mat_train
y_train = label_ppg_train
X_test = ppg_mat_test
y_test = label_ppg_test

from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [ ]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=5, gamma=0.5).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

### SVM PPG FILTERED PCA

In [ ]:
ppg = pd.read_csv("../DATA_CSV/PPG_filtered_noscaled_cammina.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ppg_df, test_ppg_df = train_test_split(ppg, test_size=0.2, stratify=ppg['Label'], random_state=123)

ppg_mat_train, label_ppg_train = create_array(train_ppg_df) 
ppg_mat_test, label_ppg_test   = create_array(test_ppg_df) 

num_components = 0.95
pca2 = PCA(n_components = num_components)
pca2.fit(ecg_mat_train)
ecg_mat_train = pca2.transform(ecg_mat_train)
ecg_mat_test = pca2.transform(ecg_mat_test)

X_train = ppg_mat_train
y_train = label_ppg_train
X_test = ppg_mat_test
y_test = label_ppg_test

from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [ ]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=5, gamma=0.5).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

### SVM PPG noFILTERED

In [ ]:
ppg = pd.read_csv("../DATA_CSV/PPG_nofiltered_noscaled_cammina.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ppg_df, test_ppg_df = train_test_split(ppg, test_size=0.2, stratify=ppg['Label'], random_state=123)

ppg_mat_train, label_ppg_train = create_array(train_ppg_df) 
ppg_mat_test, label_ppg_test   = create_array(test_ppg_df) 

X_train = ppg_mat_train
y_train = label_ppg_train
X_test = ppg_mat_test
y_test = label_ppg_test

from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [ ]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2,max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=5, gamma=0.5).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))

### SVM PPG FILTERED PCA

In [ ]:
ppg = pd.read_csv("../DATA_CSV/PPG_nofiltered_noscaled_cammina.csv",names=["index","Signal","Label"],skiprows=1).drop(columns="index")

train_ppg_df, test_ppg_df = train_test_split(ppg, test_size=0.2, stratify=ppg['Label'], random_state=123)

ppg_mat_train, label_ppg_train = create_array(train_ppg_df) 
ppg_mat_test, label_ppg_test   = create_array(test_ppg_df) 

num_components = 0.95
pca2 = PCA(n_components = num_components)
pca2.fit(ecg_mat_train)
ecg_mat_train = pca2.transform(ecg_mat_train)
ecg_mat_test = pca2.transform(ecg_mat_test)

X_train = ppg_mat_train
y_train = label_ppg_train
X_test = ppg_mat_test
y_test = label_ppg_test

from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [ ]:
# parameters for linear SVM
parameters = {'kernel' : ['linear'] , 'C': [0.001, 0.01, 0.1, 1, 10, 100]}
  
svc = SVC()
clf = GridSearchCV(svc, parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR LINEAR KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("All scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for poly with degree 2 kernel
parameters = {'C': [0.05, 0.5, 5],'gamma':[0.05,0.5,5.]}

#run SVM with poly of degree 2 kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='poly', degree=2, max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR POLY DEGREE=2 KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# parameters for rbf SVM
parameters = {'C': [0.5, 5, 50, 500],'gamma':[0.005, 0.05, 0.5,5]}

#run SVM with rbf kernel

svc = SVC()
clf = GridSearchCV(SVC(kernel='rbf', max_iter = 1000), parameters, cv=4, return_train_score=True)
clf.fit(X_train, y_train)

print ('RESULTS FOR rbf KERNEL')

print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
df=pd.DataFrame(clf.cv_results_)
df[['params', 'mean_test_score']]

In [ ]:
# execution time
t = []
for i in range(100):
    start_time = time.time()
    best_SVM = SVC(kernel='rbf', C=5, gamma=0.5).fit(X_train,y_train)
    t.append(time.time() - start_time)
print('Execution time = ', np.mean(t))
print('Highest score = ', best_SVM.score(X_test,y_test))